In [26]:
# imports
from datetime import datetime
from models import Base, Product, Store, Type_product, Contry, Currency, Rol, User
from sqlalchemy import insert, update, create_engine, inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os
import pandas as pd
import ast

In [27]:
class Connection():
    url = f"sqlite:///example_matcher2.sqlit"
    engine = create_engine(url)
    Session = sessionmaker(bind=engine)
    session = Session()

In [28]:
# funcion para crear las tablas del model
def create_tables()->None:
    
    conection = Connection()
    with conection.engine.connect() as con:
        Base.metadata.create_all(conection.engine, checkfirst=True)

In [29]:
def create(model, data: list | dict = None) -> dict | None: # podemos hacer que la funcion detecte el modelo solo nanalizando el diccionario
    
    len_data = len(data)
    
    def internal_function_verificate(dt):
        error_names = []
        for key in dt.keys():
            if key not in model.__column__:
                error_names.append(key)

        if len(error_names) == 1: return False, f"El campo '{''.join(error_names)}' esta mal escrito."
        
        elif len(error_names) > 1: return False, f"Los campos '{','.join(error_names)}' estan mal escritos."
        
        else: return True, ""
        
    dict_error = []
    
    if type(data) is list:
        
        msm_error = []
        index_error = []
        
        for index, dt in enumerate(data):
            vf_bool, print_string = internal_function_verificate(dt=dt)
            if not(vf_bool): 
                msm_error.append(print_string)
                index_error.append(index)
        
        if len(index_error) == len(data):
            print("El formato ingresado esta completamente malo verifica bien que este bien escrito el nombre de los campos")
            return False
        
        if len(index_error) >= 1:
            index_error.reverse()
            for index in index_error:
                dict_error.append(data.pop(index))

            print(f"de {len_data}, {len(index_error)} estan malos, los cuales sus index son: {index_error}")
            print(f"del resto {len(data)} su estructura esta bien asi que se procedera a crear en la base de datos.")

            
    else: 
        vf_bool, print_string = internal_function_verificate(dt=data)
        if not(vf_bool):
            print(print_string)
            return vf_bool


    conection = Connection()
    with conection.engine.connect() as con:
        if type(data) is list:
            try:
                conection.session.execute(insert(model), data)
                model.__str_create__(data = data)
            except TypeError as e:
                model.__str_create__(data = data, error = False)
                print(e)
        else:
            try:
                conection.session.execute(insert(model), data)
                model.__str_create__(data = data)
            except:
                model.__str_create__(data = data, error = False)
            
        conection.session.commit()
    
    if len(dict_error) >= 1:
        print("Se te va a retornar los diccionarios que estan mal estructurados para que los puedas arreglas e ingresar de nuevo")
        return dict_error

In [30]:
def read(model, column_name_list: list | str = None, column_name_search: str = None, search : str = None, debug = False) -> list | dict:
    
    if column_name_list == None: # para verificar que column_name_list este bien escrito
        column_name_list = "*"
    else:
        if type(column_name_list) is str:
            if column_name_list not in   model.__column_db__:
                print(f"{column_name_list} esta mal escrito.")
                return False
        elif type(column_name_list) is str: 
            error_column = []
            for name in column_name_list:
                if name not in  model.__column_db__:
                    error_column.append(name)
            if len(error_column) >= 1:
                for error_name in error_column:
                    print(print(f"{error_name} esta mal escrito."))
                return False
    if column_name_search != None:
        if column_name_search not in model.__column_db__: 
            print("EL nombre de la columna donde se va realizar la busqueda no existe no existe o este mal escrita.")
            return False
    
    query = f"SELECT {column_name_list} FROM {model.__tablename__}"

    
    conection = Connection()
    with conection.engine.connect() as con:
        if column_name_search == None and search == None: # si no se le astablece un paramtro de busqueda listara todos los elementos de la tabla y retornara un dataFrame
            data = conection.session.execute(text(query)).fetchall()
            data_list = []
            for index in range(0, len(data)):
                data_dict = {key:value for key,value in zip(model.__column__, data[index])}
                data_list.append(data_dict)
            return pd.DataFrame(data_list)
        
        else:
            
            data = conection.session.execute(text(f"{query} WHERE {column_name_search} = '{search}'")).fetchall()
            if len(data) >= 2:
                data_list = []
                for index in range(0, len(data)):
                    data_dict = {key:value for key,value in zip(model.__column__, data[index])}
                    data_list.append(data_dict)
                return True, data_list
            
            elif len(data) == 1:
                return True, {key:value for key,value in zip(model.__column__, data[0])}
            
            elif len(data) == 0:
                print("Lo que esta buscando no existe o esta mal escrito, intente de nuevo")
                return False, False
            
            
        

In [24]:
def update(model, data: list | dict = None, param: str = None, param_search: str = None) -> None:
    
    if param_search == None:
        param_search = data[param]
    
    status, data_to_compare = read(model=model, column_name_search=param, search=param_search)
    
    data_to_compare = ast.literal_eval(str(data_to_compare))
    
    dict_update = {}
    dict_update[model.__column__[0]] = data_to_compare[model.__column__[0]]
    
    for compara, toCompare in zip(data.keys(), list(data_to_compare.keys())[1:]):
        
        if type(data[compara]) is str and type(data_to_compare[toCompare]) is str:
            if data[compara] != data_to_compare[toCompare] is str:
                dict_update[compara] = data[compara]
        
        elif type(data[compara]) is list and type(data_to_compare[toCompare]) is list:
            for str_compara, str_to_compare in zip(data[compara], data_to_compare[toCompare]):
                if str_compara != str_to_compare:
                    dict_update[compara] = data[compara]
        
        elif type(data[compara]) is dict and type(data_to_compare[toCompare]) is dict:    
            dict_update[compara] = data[compara] 
    
    print(dict_update)


data2 = {'store_id_fk': 1, 'product_type_id_fk': 1, 'currency_id_fk': 1, 'link_produc': 'https://tijerazo.com/products/black-ceramic-pour-over-pot?variant=397485936476812', 'title': 'Black Ceramic Pour-Over Pot', 'description': 'You will have ultimate control over your coffee', 'product_brand': 'Adidas', 'img_urls': ['https://assets.ajio.com/medias/sys_master/root/20210403/8Hr3/606872837cdb8c1f1475e982/-473Wx593H-461211848-white-MODEL.jpg', 'https://assets.ajio.com/medias/sys_master/root/20210403/xELq/606872747cdb8c1f1475e7f5/-473Wx593H-461211848-white-MODEL2.jpg', 'https://assets.ajio.com/medias/sys_master/root/20210403/We00/606872747cdb8c1f1475e7e8/-473Wx593H-461211848-white-MODEL3.jpg', 'https://assets.ajio.com/medias/sys_master/root/20210403/AXUd/606872747cdb8c1f1475e7f7/-473Wx593H-461211848-white-MODEL4.jpg', 'https://assets.ajio.com/medias/sys_master/root/20210403/NQYn/606872837cdb8c1f1475e8fa/-473Wx593H-461211848-white-MODEL5.jpg'], 'img_direction_folder': ['tijerazo-Canada-Camisas--473Wx593H-461211848-white-MODEL.jpg', 'tijerazo-Canada-Camisas--473Wx593H-461211848-white-MODEL2.jpg', 'tijerazo-Canada-Camisas--473Wx593H-461211848-white-MODEL3.jpg', 'tijerazo-Canada-Camisas--473Wx593H-461211848-white-MODEL4.jpg', 'tijerazo-Canada-Camisas--473Wx593H-461211848-white-MODEL5.jpg'], 'price': 467, 'discount_price': 450, 'has_discount': True, 'characteristics': {'color': 'blue', 'size': '27 cm', 'product_genre': 'unisex', 'calification': '5', 'number_comments': '1402'}}
update(Product,  param="link_produc", data=data2)

{'product_id': 3}


In [31]:
def load_collector(json_file: str = None) -> list:
    
    dataFrame = pd.read_json(json_file)
    
    collector = []
    
    for index in range(0, len(dataFrame)):
        
        # verifica que exista la tienda
        store_id = ""
        status_store, store = read(Store, column_name_search="store_name" ,search=dataFrame['store_name'][index])
        if not(status_store):
            contry_id = ""
            status_contry, contry = read(Contry, column_name_search="contry_name" ,search=dataFrame['contry'][index])
            if not(status_contry):
                create(Contry, {"contry_name":dataFrame['contry'][index]})
                status_contry, contry = read(Contry, column_name_search="contry_name" ,search=dataFrame['contry'][index])
            contry_id = contry["contry_id"]
            
            create(Store, {"contry_id_fk":contry_id, "store_name": dataFrame['store_name'][index], "link_store": dataFrame['product_type'][index]})
            status_store, store = read(Store, column_name_search="store_name" ,search=dataFrame['store_name'][index])
            store_id = store["store_id"]
        store_id = store["store_id"]
        
        
        # verifica que exista el tipo de producto
        product_type_id = ""
        status_product_type, product_type= read(Type_product,  column_name_search="product_type_name" ,search=dataFrame['product_type'][index])
        if not(status_product_type):
            create(Type_product, {"product_type_name":dataFrame['product_type'][index]})
            status_product_type, product_type= read(Type_product,  column_name_search="product_type_name" ,search=dataFrame['product_type'][index])
            product_type_id = product_type["product_type_id"]
        product_type_id = product_type["product_type_id"]
        
        
        # verifica que exista el tipo de currency
        currency_id = ""
        status_currency, currency= read(Currency,  column_name_search="currency_name" ,search=dataFrame['price'][index]["currency"])
        if not(status_currency):
            create(Currency, {"currency_name":dataFrame['price'][index]["currency"], "symbol":dataFrame['price'][index]["symbol"]})
            status_currency, currency= read(Currency,  column_name_search="currency_name" ,search=dataFrame['price'][index]["currency"])
            currency_id = currency["currency_id"]
        currency_id = currency["currency_id"]

        
        data = {
        "store_id_fk": store_id,
        "product_type_id_fk": product_type_id,
        "currency_id_fk": currency_id,
        "link_produc": dataFrame['link'][index],
        "title": dataFrame['title'][index], 
        "description": dataFrame['description'][index], 
        "product_brand": dataFrame['product_brand'][index], 
        "img_urls": dataFrame['img_urls'][index],
        "img_direction_folder": dataFrame['img_direction_folder'][index], 
        "price": dataFrame['price'][index]["value"],
        "discount_price": dataFrame['discount_price'][index]["value"],
        "has_discount":(dataFrame["price"][index]["value"] - dataFrame["discount_price"][index]["value"]) != 0,
        "characteristics": dataFrame['characteristics'][index]
        }

        collector.append(data)

    return collector

In [32]:
data2 = load_collector("ejemplo_load_collector.json")
create(Product, data2)

OperationalError: (sqlite3.OperationalError) no such table: stores
[SQL: SELECT * FROM stores WHERE store_name = 'tijerazo']
(Background on this error at: https://sqlalche.me/e/20/e3q8)